In [12]:
%load_ext autoreload
%autoreload 2

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [14]:
from __future__ import print_function, division
import planet4 as p4
import pandas as pd
from planet4 import markings
from planet4.get_data import get_current_database_fname
from planet4 import clustering
import os
from os.path import join as pjoin
HOME = os.environ['HOME']

In [15]:
dbfile = get_current_database_fname()
store = pd.HDFStore(dbfile)
store

<class 'pandas.io.pytables.HDFStore'>
File path: /Users/klay6683/data/planet4/2015-03-08_planet_four_classifications_queryable.h5
/df            frame_table  (typ->appendable,nrows->12554247,ncols->21,indexers->[index],dc->[classification_id,image_id,image_name,user_name,marking,acquisition_date,local_mars_time])

In [28]:
image_names = store.select_column('df', 'image_name').unique()

In [41]:
from IPython.parallel import Client
client = Client()

In [42]:
dview = client.direct_view()
lview = client.load_balanced_view()

In [43]:
%%px
import pandas as pd
from planet4 import clustering, markings
from os.path import join as pjoin
import os
HOME = os.environ['HOME']

In [48]:
def do_clustering(p4img, fans):
    if fans:
        reduced = clustering.perform_dbscan(p4img.get_fans(), fans=fans)
    else:
        reduced = clustering.perform_dbscan(p4img.get_blotches(), fans=fans)
    if reduced is None:
        return None
    series = [cluster.data for cluster in reduced]
    n_members = [cluster.n_members for cluster in reduced]
    df = pd.DataFrame(series)
    df['image_id'] = p4img.imgid
    df['n_members'] = n_members
    return df
    
def process_image_name(image_name):
    dirname = pjoin(HOME, 'data/planet4/reduced')
    blotchfname = pjoin(dirname, image_name+'_reduced_blotches.hdf')
    fanfname = pjoin(dirname, image_name+'_reduced_fans.hdf')
    if os.path.exists(blotchfname) and\
            os.path.exists(fanfname):
        return image_name+' already done.'
    data = pd.read_hdf(dbfile, 'df', where="image_name="+image_name)
    img_ids = data.image_id.unique()
    blotches = []
    fans = []
    for img_id in img_ids:
        p4img = markings.ImageID(img_id)
        blotches.append(do_clustering(p4img, fans=False))
        fans.append(do_clustering(p4img, fans=True))
    blotches = pd.concat(blotches, ignore_index=True)
    blotches.to_hdf(blotchfname, 'df')
    fans = pd.concat(fans, ignore_index=True)
    fans.to_hdf(fanfname, 'df')
    return image_name

In [49]:
dview.push({'do_clustering':do_clustering,
            'dbfile':dbfile})

<AsyncResult: finished>

In [55]:
result = lview.map_async(process_image_name, image_names)

In [54]:
for res in result:
    print(res)

ESP_011544_0985
ESP_021684_0985
ESP_011697_0980


In [59]:
import time
import sys
while not result.ready():
    print("{:.1f} %".format(100*result.progress/len(image_names)))
    sys.stdout.flush()
    time.sleep(30)

2.5 %
3.2 %
3.2 %
3.9 %
4.2 %
4.9 %
5.6 %
6.1 %
6.6 %
7.4 %
7.8 %
8.6 %
9.3 %
9.8 %
10.8 %
11.5 %
12.0 %
12.7 %
13.0 %
13.7 %
14.2 %
15.0 %
16.2 %
17.4 %
17.9 %
18.6 %
18.9 %
19.9 %
21.1 %
21.6 %
22.5 %
23.0 %
23.5 %
24.8 %
26.0 %
27.0 %
27.5 %
28.2 %
28.4 %
29.4 %
30.4 %
31.1 %
32.4 %
32.8 %
33.8 %
34.8 %
36.5 %
37.0 %
38.0 %
39.5 %
40.2 %
41.4 %
42.2 %
43.4 %
44.6 %
45.8 %
47.1 %
48.3 %
49.8 %
51.0 %
52.0 %
53.4 %
54.4 %
54.9 %
55.4 %
55.6 %
56.4 %
56.6 %
57.1 %
57.6 %
58.1 %
58.6 %
59.3 %
59.8 %
60.8 %
61.3 %
62.3 %
62.7 %
63.5 %
64.5 %
65.9 %
66.4 %
67.2 %
68.4 %
69.6 %
70.3 %
71.3 %
72.8 %
74.3 %
75.0 %
76.2 %
77.0 %
77.9 %
78.9 %
80.1 %
81.4 %
82.4 %
83.8 %
85.5 %
87.3 %
89.0 %
90.2 %
92.2 %
94.6 %
95.6 %
97.1 %
98.3 %
99.8 %


In [2]:
reducedfiles = !ls ~/data/planet4/reduced
nooffiles = len(reducedfiles)
print("Produced", nooffiles, "files.")

Produced 814 files.


In [4]:
from planet4.get_data import is_catalog_production_good

In [5]:
is_catalog_production_good()

True